In [11]:
import pandas as pd
from sklearn import preprocessing
import tensorflow as tf

# display all columns
pd.options.display.max_columns = None

In [12]:
# Utils
from sklearn import preprocessing
def encode_text_index(df, name):
    le = preprocessing.LabelEncoder()
    df[name] = le.fit_transform(df[name])
    return le.classes_
# Encode a numeric column as zscores
def encode_numeric_zscore(df, name, mean=None, sd=None):
    if mean is None:
        mean = df[name].mean()

    if sd is None:
        sd = df[name].std()

    df[name] = (df[name] - mean) / sd
# Encode text values to dummy variables(i.e. [1,0,0],[0,1,0],[0,0,1] for red,green,blue)
def encode_text_dummy(df, name):
    dummies = pd.get_dummies(df[name])
    for x in dummies.columns:
        dummy_name = f"{name}-{x}"
        df[dummy_name] = dummies[x]
    df.drop(name, axis=1, inplace=True)
# Convert a Pandas dataframe to the x,y inputs that TensorFlow needs
def to_xy(df, target):
    result = []
    for x in df.columns:
        if x != target:
            result.append(x)
    # find out the type of the target column.  Is it really this hard? :(
    target_type = df[target].dtypes
    target_type = target_type[0] if hasattr(
        target_type, '__iter__') else target_type
    # Encode to int for classification, float otherwise. TensorFlow likes 32 bits.
    if target_type in (np.int64, np.int32):
        # Classification
        dummies = pd.get_dummies(df[target])
        return df[result].values.astype(np.float32), dummies.values.astype(np.float32)
    # Regression
    return df[result].values.astype(np.float32), df[[target]].values.astype(np.float32)

The history saving thread hit an unexpected error (OperationalError('database is locked')).History will not be written to the database.


In [13]:
path = "/Users/nguyenquangminh/Desktop/Thesis/Dataset/CICDataset/CIC-IDS-2017/TrafficLabelling /Wednesday-workingHours.pcap_ISCX.csv"
dataset = pd.read_csv(filepath_or_buffer=path)
print(dataset)
print(dataset[" Label"].unique())

                                         Flow ID       Source IP  \
0         192.168.10.14-209.48.71.168-49459-80-6   192.168.10.14   
1         192.168.10.3-192.168.10.17-389-49453-6   192.168.10.17   
2          192.168.10.3-192.168.10.17-88-46124-6   192.168.10.17   
3         192.168.10.3-192.168.10.17-389-49454-6   192.168.10.17   
4          192.168.10.3-192.168.10.17-88-46126-6   192.168.10.17   
...                                          ...             ...   
692698    192.168.10.3-192.168.10.14-53-51114-17   192.168.10.14   
692699    192.168.10.3-192.168.10.16-53-24054-17   192.168.10.16   
692700  192.168.10.51-23.208.163.130-58030-443-6  23.208.163.130   
692701    192.168.10.3-192.168.10.14-53-51694-17   192.168.10.14   
692702    192.168.10.3-192.168.10.14-53-57949-17   192.168.10.14   

         Source Port  Destination IP   Destination Port   Protocol  \
0              49459   209.48.71.168                 80          6   
1              49453    192.168.10.3       

In [14]:
dataset.drop(columns=['Flow ID', ' Source IP', ' Source Port', ' Destination IP', ' Timestamp'], inplace=True)

In [15]:
for column in dataset.columns:
    if column != " Label":
        encode_numeric_zscore(dataset, column)
    else:
        encode_text_index(dataset, column)

In [16]:
nan_columns = dataset.columns[dataset.isna().any()]
print(nan_columns)
dataset.drop(columns=nan_columns, inplace=True)

Index(['Flow Bytes/s', ' Flow Packets/s', ' Bwd PSH Flags', ' Fwd URG Flags',
       ' Bwd URG Flags', ' CWE Flag Count', 'Fwd Avg Bytes/Bulk',
       ' Fwd Avg Packets/Bulk', ' Fwd Avg Bulk Rate', ' Bwd Avg Bytes/Bulk',
       ' Bwd Avg Packets/Bulk', 'Bwd Avg Bulk Rate'],
      dtype='object')


In [7]:
import pandas as pd
import io
import requests
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn import metrics
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.callbacks import EarlyStopping

# Break into X (predictors) & y (prediction)
x, y = to_xy(dataset,' Label')

# Create a test/train split.  25% test
# Split into train/test
x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=0.25, random_state=42)

# Create neural net
model = Sequential()
model.add(Dense(10, input_dim=x.shape[1], kernel_initializer='normal', activation='relu'))
model.add(Dense(50, input_dim=x.shape[1], kernel_initializer='normal', activation='relu'))
model.add(Dense(10, input_dim=x.shape[1], kernel_initializer='normal', activation='relu'))
model.add(Dense(1, kernel_initializer='normal'))
model.add(Dense(y.shape[1],activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=5, verbose=1, mode='auto')
model.fit(x_train,y_train,validation_data=(x_test,y_test),callbacks=[monitor],verbose=2,epochs=1000)

Epoch 1/1000
16236/16236 - 14s - loss: 0.1534 - val_loss: 0.0755 - 14s/epoch - 878us/step
Epoch 2/1000
16236/16236 - 14s - loss: 0.0719 - val_loss: 0.0660 - 14s/epoch - 874us/step
Epoch 3/1000
16236/16236 - 14s - loss: 0.0624 - val_loss: 0.0649 - 14s/epoch - 855us/step
Epoch 4/1000
16236/16236 - 14s - loss: 0.0580 - val_loss: 0.0536 - 14s/epoch - 873us/step
Epoch 5/1000
16236/16236 - 14s - loss: 0.0544 - val_loss: 0.0519 - 14s/epoch - 869us/step
Epoch 6/1000
16236/16236 - 14s - loss: 0.0515 - val_loss: 0.0493 - 14s/epoch - 871us/step
Epoch 7/1000
16236/16236 - 15s - loss: 0.0484 - val_loss: 0.0569 - 15s/epoch - 906us/step
Epoch 8/1000
16236/16236 - 14s - loss: 0.0465 - val_loss: 0.0490 - 14s/epoch - 852us/step
Epoch 9/1000
16236/16236 - 14s - loss: 0.0451 - val_loss: 0.0443 - 14s/epoch - 853us/step
Epoch 10/1000
16236/16236 - 14s - loss: 0.0434 - val_loss: 0.0460 - 14s/epoch - 853us/step
Epoch 11/1000
16236/16236 - 14s - loss: 0.0424 - val_loss: 0.0456 - 14s/epoch - 854us/step
Epoch 12

In [9]:
# Measure accuracy
pred = model.predict(x_test)
pred = np.argmax(pred,axis=1)
y_eval = np.argmax(y_test,axis=1)
score = metrics.accuracy_score(y_eval, pred)
print("Validation score: {}".format(score))

5412/5412 [==============================] - 3s 555us/step
Validation score: 0.9938675105095395


In [8]:
x_train

array([[-0.35650274, -0.64264554,  1.3503433 , ..., -0.10568635,
         1.6391236 ,  1.6772817 ],
       [-0.35650274, -0.64264554,  1.6733824 , ..., -0.10568635,
         2.0003228 ,  2.042329  ],
       [-0.35821947,  1.5530529 , -0.49380755, ..., -0.10568635,
        -0.40759438, -0.3912419 ],
       ...,
       [-0.35650274, -0.64264554,  1.64712   , ..., -0.10568635,
         1.96914   ,  2.0108142 ],
       [-0.35821947,  1.5530529 , -0.6547482 , ..., -0.10568635,
        -0.5852399 , -0.5707801 ],
       [ 3.3589182 , -0.64264554,  1.3087351 , ..., -0.10568635,
         1.5689626 ,  1.6063732 ]], dtype=float32)

In [10]:
y_train

array([[0., 0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       ...,
       [0., 0., 1., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.]], dtype=float32)